In [0]:
spark.conf.set("fs.azure.account.auth.type.zee.dfs.core.windows.net", "SAS")
spark.conf.set("fs.azure.sas.token.provider.type.zee.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set("fs.azure.sas.fixed.token.zee.dfs.core.windows.net", "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2023-08-03T21:41:56Z&st=2023-08-03T13:41:56Z&spr=https&sig=QMrb%2FlACHhwjJ1LNAcIXlqnjIidoinC6CvOaPAkh9JI%3D")


In [0]:
driver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

database_host = "ankitbhai.database.windows.net"
database_port = "1433" # update if you use a non-default port
database_name = "Ankitbhai"
table = "dbo.employees"
user1 = "ankitbhai"
password1 = "Ankit123"

url = f"jdbc:sqlserver://{database_host}:{database_port};database={database_name}"

In [0]:
df = spark.read.format('jdbc').option('url',url).option("dbtable", table).option("user", user1).option("password", password1).load()

In [0]:
df.display()

employee_id,first_name,last_name,age,department
1,John,Doe,30,HR
2,Jane,Smith,28,IT
3,Michael,Johnson,35,Finance
4,Emily,Williams,25,Marketing


In [0]:
(spark.readStream.format('jdbc').option('url',url).option("dbtable", table).option("user", user1).option("password", password1).load()
      .writeStream
        .option("checkpointLocation", "dbfs:/mnt/demo/orders_checkpoint")
        .table("orders_updates")
)

In [0]:
path= "/mnt/ankit/POC/"


files = dbutils.fs.ls(f"{path}")
display(files)

path,name,size,modificationTime
dbfs:/mnt/ankit/POC/Water_Quality_Testing.csv,Water_Quality_Testing.csv,13263,1686548110000


In [0]:
(spark.readStream.format("csv").option("header", "true").option("inferSchema", "true").load(f"{path}").writeStream
        .option("checkpointLocation", "dbfs:/mnt/ankit/orders_checkpoint/check1/orders_checkpoint")
        .table("dbfs:/mnt/ankit/final/")
)

In [0]:
df2=spark.readStream.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/mnt/ankit/POC/")

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
File <command-3445791470341651>:1
----> 1 df2=spark.readStream.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/mnt/ankit/POC/")

File /databricks/spark/python/pyspark/sql/streaming/readwriter.py:275, in DataStreamReader.load(self, path, format, schema, **options)
    270     if type(path) != str or len(path.strip()) == 0:
    271         raise ValueError(
    272             "If the path is provided for stream, it needs to be a "
    273             + "non-empty string. List of paths are not supported."
    274         )
--> 275     return self._df(self._jreader.load(path))
    276 else:
    277     return self._df(self._jreader.load())

File /databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py:1321, in JavaMember.__call__(self, *args)
   1315 command = proto.CALL_COMMAND_NAME +\
   13

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Define the schema for the DataFrame
schema = StructType([
    StructField("PassengerId", IntegerType(), True),
    StructField("Survived", IntegerType(), True),
    StructField("Pclass", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("Sex", StringType(), True),
    StructField("Age", DoubleType(), True),
    StructField("SibSp", IntegerType(), True),
    StructField("Parch", IntegerType(), True),
    StructField("Ticket", StringType(), True),
    StructField("Fare", DoubleType(), True),
    StructField("Cabin", StringType(), True),
    StructField("Embarked", StringType(), True),
])

In [0]:
df = spark.readStream.format("csv").schema(schema).option("header", True).load("abfs://csv@zee.dfs.core.windows.net/Gold/")

df.display()

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,null,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,null,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,null,S
6,0,3,"Moran, Mr. James",male,null,0,0,330877,8.4583,null,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.075,null,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,null,S
11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7,G6,S


In [0]:
df.writeStream.format("parquet").outputMode("append").option("path", "abfs://csv@zee.dfs.core.windows.net/final/").option("checkpointLocation", "abfs://csv@zee.dfs.core.windows.net/orders_checkpoint/").start()

Out[69]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f01f9125130>